# Consideraciones Importantes

Hasta ahora se han creado dos contenedores, uno para la BD con MONGO, y otro para la API con FASTAPI. Sin embargo como se ha trabajado desde varios computadores, es importante tener en cuenta que la BD esta alojada en un Volumen de Docker, que se llama `dbAppTienda`. Si por alguna razon no se ha creado en un computador o en otro, crearla con datos de prueba. Y la manera correcta de levantar el servicio es:

```sh
docker run --detach --name tuTiendaDB --mount src=dbAppTienda,dst=/data/db --publish 27017:27017 --network platzinet mongo:5.0
```
Para evitar incompatibilidades, se usara de aqui en adelante *mongo 5.0*.

- `docker volume create dbAppTienda` si no se ha creado el volumen. 

Para la API, no se usara por ahora contenedor. Se iniciara directamente desde la consola `uvicorn`.

`uvicorn main:app --reload`

Se creara una nueva coleccion en la BD, llamada `db-tienda`, con una unica coleccion, llamada `clients`, ver `insert_clients.js` donde esta el script.

## Seguir avanzando

Para seguir avanzando en un codigo eficiente, introduciremos dos conceptos fundamentales, el primero es Eventos de Ciclo de Vida, y el segundo Declaracion Tipo Opcional

## Eventos de Ciclo de Vida

FastAPI, al igual que otros frameworks web basados en ASGI (como Starlette, en el que se basa FastAPI), proporciona "eventos de ciclo de vida" que te permiten ejecutar código en momentos específicos de la vida de tu aplicación.

Los eventos de ciclo de vida son un concepto fundamental en el desarrollo de software, especialmente en aplicaciones que manejan recursos externos o que necesitan realizar acciones en momentos específicos de su ejecución. No son exclusivos de FastAPI o Python; son una idea universal que encontrarás en la mayoría de los frameworks y lenguajes modernos.

### ¿Qué son los Eventos de Ciclo de Vida?

En términos generales, los eventos de ciclo de vida son ganchos o puntos de extensión predefinidos en la línea de tiempo de una aplicación, un componente, un servicio o incluso un objeto. Estos ganchos te permiten ejecutar código personalizado en fases clave, como:

- Inicialización: Cuando algo se crea o se pone en marcha.

- Inicio: Cuando un servicio o aplicación comienza a estar disponible para operar.

- Actualización/Cambio: Cuando el estado de algo cambia.

- Pausa/Suspensión: Cuando una operación se detiene temporalmente.

- Reanudación: Cuando una operación se reinicia después de una pausa.

- Destrucción/Apagado: Cuando algo finaliza su ejecución y libera recursos.

### Eventos Startup y Shutdown

#### Evento startup (Inicialización)

El evento `startup` en FastAPI se refiere a las acciones que tu aplicación debe realizar una única vez cuando se está iniciando, antes de que esté lista para procesar cualquier solicitud HTTP entrante.

##### ¿Cuándo se ejecuta?
- Cuando ejecutas tu aplicación FastAPI (por ejemplo, con ``uvicorn main:app --reload``).

- FastAPI detecta que el servidor se está levantando.

- Antes de que el servidor abra sus puertos y empiece a escuchar peticiones, ejecuta todas las funciones decoradas con ``@app.on_event("startup")``

Se usa comunmente para: extablecer conexiones a bases de datos, cargar configuraciones globales como variables de entorno, etc

#### Evento shutdown (Apagado)

El evento ``shutdown`` en FastAPI se refiere a las acciones que tu aplicación debe realizar una única vez cuando se está cerrando, antes de que el proceso termine por completo.

- Cuando el servidor FastAPI recibe una señal para apagarse (por ejemplo, presionas ``Ctrl+C`` en la terminal donde corre ``uvicorn``, o un orquestador como Docker/Kubernetes envía una señal de terminación).

- FastAPI detiene la aceptación de nuevas solicitudes.

- Antes de que el proceso de la aplicación finalice, ejecuta todas las funciones decoradas con ``@app.on_event("shutdown")``.

Se usan comunmente para cerrar conexiones a bases de datos, vaciar logs, liberarar recursos del sistema entre otros. 

En resumen, los eventos ``startup`` y ``shutdown`` son tus mejores amigos para manejar el ciclo de vida de los recursos de tu aplicación, garantizando que todo se inicialice correctamente y se limpie de forma segura.

## Declaración de Tipo Opcional

En programación, una Declaración de Tipo Opcional significa que una variable, un parámetro de función o un valor de retorno *puede ser de un tipo específico o puede ser "nada"* (generalmente representado por ``None`` en Python, ``null`` en JavaScript/Java/C#).

En Python, esto se expresa usando ``Optional[Tipo]`` (por ejemplo, ``Optional[int]``).

### ¿Por qué es Necesario en Cierto Tipo de Situaciones?

La Declaración de Tipo Opcional es esencial en situaciones donde la ausencia de un valor es una posibilidad legítima y debe ser manejada de forma explícita, no como un error inesperado. 

1. Recursos Externos que Podrían Fallar o No Estar Presentes: *Conexiones a Bases de Datos o APIs Externas:* Como tu ejemplo de ``db_client``. Una aplicación no puede asumir que siempre tendrá una conexión activa a la DB. La conexión puede no establecerse al inicio, o puede perderse durante la ejecución. El ``Optional`` te fuerza a verificar si la conexión existe antes de usarla.

2. Búsquedas de Datos (Cuando el Elemento Podría No Existir):Ejemplo: Buscar un usuario por ID. Si el ID no existe, la función debe retornar "nada" (e.g., ``None``), no un usuario incompleto o un error que cause un crasheo. El tipo de retorno de la función sería ``Optional[UserObject]``.


### Implementando en Codigo 

Se elimina el *endpoint* : *check_db_connection_status*, ya no es necesario. 

```py
from fastapi import FastAPI, HTTPException, Path
from fastapi.responses import HTMLResponse, JSONResponse
from motor.motor_asyncio import AsyncIOMotorClient
from datos import CLIENTS as clients
from typing import Optional 👈

# Importar asynccontextmanager para los eventos lifespan
from contextlib import asynccontextmanager


# --- Configuración de MongoDB ---
# MONGO_DETAILS = "mongodb://tuTiendaDB:27017/"
MONGO_DETAILS = "mongodb://localhost:27017/"


# Guarda la conexión global a MongoDB; es None inicialmente y se establece al inicio de la app.
db_client : Optional[AsyncIOMotorClient] = None 👈


# Create a FastAPI application instance
app = FastAPI()
app.title = "Prototipo Plataforma de Comercializacion"
app.version = "0.0.2"


@app.on_event("startup")
async def startup_db_client():
    try:
        global db_client 👈
        db_client = AsyncIOMotorClient(MONGO_DETAILS) 👈
        print(f'Conexion Exitosa a la BD: {MONGO_DETAILS} ')
    except Exception as e:
        raise HTTPException(status_code=500, detail={"message":"Error de Servidor BD no Disponible"})
    

@app.on_event("shutdown")
async def shutdown_db_client():
    if db_client:
        db_client.close()
        print('Se ha cerrado la conexion') 👈
```

```py
db_client : Optional[AsyncIOMotorClient] = None
```

Esta línea le dice a Python y a las herramientas de análisis de código (como Pylance) que la variable ``db_client`` puede contener una conexión a la base de datos de MongoDB (``AsyncIOMotorClient``) o puede no contener nada (``None``). Se inicializa a ``None`` para indicar que, al principio, la conexión aún no se ha establecido.



### BUGS

Habia un bug, que cuando se cerraba la aplicacion con CTRL+C no imprimia el mensaje de que se hubiera cerrado la conexion. Que se corrigio de la siguiente manera:

1. La línea ``db_client : Optional[AsyncIOMotorClient] = None`` crea una variable db_client en el ámbito global de tu módulo.

2. Dentro de ``startup_db_client``, al hacer ``db_client = AsyncIOMotorClient(MONGO_DETAILS)`` sin usar la palabra clave ``global``, Python interpreta esto como la creación de una nueva variable local llamada ``db_client`` dentro de la función ``startup_db_client``.

3. Esta variable ``db_client local`` se inicializa con la conexión de MongoDB

4. Cuando ``startup_db_client`` termina de ejecutarse, la variable ``db_client`` local desaparece (sale de ámbito).

5. La variable ``db_client`` **global** (la que está fuera de la función) nunca fue modificada y seguía siendo None

Si ahora se reinicia la aplicacion, mostrara los mensajes indicados, y por ahora no hay mas bugs. 

![image](https://imgur.com/CQvC3TO.png)

## Encontrar un elemento por su ID dentro de la BD de Mongo

Se necesita ahora si buscar recursos externos, como es nuestra base de datos en mongo, seguiremos manteninedo ``datos.py` para algunas pruebas, empezaremos a crear una funcion basica para el endpoint, y le iremos agregando complejidad:

Recordemos que para buscar datos desde `mongosh`, usamos el siguiente query:

```js
db-tienda> db.clients.findOne({'id':2})
```

Y el resultado es:

```js
{
  _id: ObjectId('6876dd7bf6e66a0696544cb2'),
  id: 2,
  name: 'Carlos',
  lastName: 'Rodríguez',
  governmentId: '7987654321',
  typeOfDocument: 'TI',
  DOB: '2000-07-22',
  address: {
    street: 'Carrera 15 # 45-80',
    city: 'Medellín',
    zipCode: '050010',
    country: 'Colombia'
  }
}
```

Usaremos el objeto `db_client`la cual guarda la conexion global a mongodb. 

```py
@app.get("/{client_id_to_find}", tags=['clients'])
async def get_client_by_id(client_id_to_find:int = Path(...,
                                                        gt=0,
                                                        description="ID del estudiante a buscar")):

    if db_client:
        collection = db_client['db-tienda'].clients 
        print((collection.name))

    return JSONResponse(status_code=200, content={'id': client_id_to_find, 'collection': collection.name})
```

En este ejemplo basico, se obtiene, la coleccion en mongodb, lo cual es un buen comienzo: 

![](https://imgur.com/TaCfQO3.png)



### Consultando Finalmente a la BD 

```py
@app.get("/{client_id_to_find}", tags=['clients'])
async def get_client_by_id(client_id_to_find:int = Path(...,
                                                        gt=0,
                                                        description="ID del estudiante a buscar")):

    if db_client:
        collection = db_client['db-tienda'].clients 
        print((collection.name))
        found_client = await collection.find_one({'id': client_id_to_find})
        print(found_client)
        if found_client:
            found_client['_id'] = str(found_client['_id']) 👈
            # return JSONResponse(status_code=200, content={'id': client_id_to_find, 'collection': collection.name, 'name': found_client['name']})
            return JSONResponse(status_code=200, content=found_client)
```

#### QUe hace `found_client['_id'] = str(found_client['_id'])`

Cuando MongoDB (a través de Motor/PyMongo) te devuelve un documento, el campo ``_id`` es un objeto bson.ObjectId. Al aplicar ``str()`` a ObjectId('...'), lo conviertes en una cadena como '6876dd7bf6e66a0696544cb1', que si se puede enviar en la respuesta. 

Una vez que comprobamos la funcion esta funcionando bien, me encargo de refactorizarla. Se agrega un dotstring. Si el estudiante no existe entonces retorna un error 404 no encontrado

```py
@app.get("/student/{client_id_to_find}", tags=['student'], status_code=200)
async def get_student_by_id(client_id_to_find: int = Path(..., 
                                                 gt=0, 
                                                 description="ID del estudiante a buscar")):
    
    """
    Busca un estudiante en DB y devuelve todos sus datos si es encontrado.
    Si el estudiante no existe, retorna un error 404 Not Found.
    """
    found_client_list = []

    for client in students:
        if client['id']==client_id_to_find:   
            return JSONResponse(status_code=200, content=client)

    message = f"estudiante con id {client_id_to_find} no existeee"
    raise HTTPException(status_code=404, detail={"message":message})
```
